<a href="https://colab.research.google.com/github/hiroki1982/Research/blob/main/Loneliness_ver2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain openai pandas
!pip install -U langchain langchain-community

In [ ]:
# LangChain × CSVベース人格読み込み＆応答傾向ロギングテンプレート（最新版）

# 1. 必要なライブラリ
!pip install -U langchain langchain-community openai pandas

# 2. ライブラリ読み込み
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
import os
from datetime import datetime
import csv

# 3. APIキーの設定（ColabのSecretsを使用）
from google.colab import userdata
api_key = userdata.get('API_KEY')

# 4. CSVファイルから人格データを読み込む
file_id = "1kfWn8yJllyufO-Il6Qk1rsFquYGemwGMGlJ5TrZjS24"
csv_url = f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv"
personas = pd.read_csv(csv_url)
personas.columns = personas.columns.str.strip().str.replace('"', '').str.replace("'", '')
personas['role'] = personas['role'].str.strip()

# 5. 利用可能な人格一覧表示と選択
print("利用可能な人格一覧:")
options = personas['role'].tolist()
for i, role in enumerate(options):
    print(f"{i}: {role}")

selected_index = int(input("使用する人格の番号を入力してください: "))
selected_role = options[selected_index]

# 6. 該当する人格の記述を取得
persona_row = personas[personas['role'] == selected_role]
if persona_row.empty:
    raise ValueError("指定された人格が見つかりませんでした")
persona_description = persona_row['description'].values[0]
print("選択された人格プロンプト:\n{persona_description}\n")

# 7. LangChainセットアップ
chat = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7, api_key=api_key)

# 8. プロンプトテンプレート作成
prompt_template = PromptTemplate(
    input_variables=["input", "persona"],
    template="""
{persona}

ユーザー: {input}
エージェント:"""
)

# 9. LLMチェーン構築
conversation = LLMChain(
    llm=chat,
    prompt=prompt_template,
    verbose=True
)

# 10. 応答ログファイル準備
log_file = f"response_log_{selected_role}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
with open(log_file, mode='w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(["timestamp", "user_input", "agent_response"])

# 11. 対話ループ開始
print("=== 人格付きLangChainエージェントとの対話開始（ロギング付き） ===")
print("(終了するには 'exit' と入力してください)")

while True:
    user_input = input("あなた: ")
    if user_input.lower() == "exit":
        print("また話しましょう。")
        break

    response = conversation.run({"input": user_input, "persona": persona_description})
    print(f"エージェント: {response}")

    with open(log_file, mode='a', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow([datetime.now().isoformat(), user_input, response])
